In [2]:
import pandas as pd
import numpy as np
import os

file = os.path.join('Resources', 'purchase_data.json')

pur_data = pd.read_json(file)

pur_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [3]:
player_count = len(pur_data['SN'].unique())

players_df = pd.DataFrame([{'Total Players': player_count}])
players_df.set_index('Total Players', inplace = True)
players_df

""
Total Players
573


In [53]:
#pur_data['Item ID'].value_counts()
#unique_items = pd.DataFrame(pur_data['Item ID'].unique())
#len(unique_items)

no_dup_items = pur_data.drop_duplicates(['Item ID'], keep = 'last')
total_unique = len(no_dup_items)
sum_purchases = pur_data['Price'].sum()
avg_price = sum_purchases/total_unique
avg_price = round(avg_price, 2)

total_pur = pur_data['Price'].count()
total_rev = round(pur_data['Price'].sum(),2)
avg_price = sum_purchases/total_pur
avg_price = round(avg_price, 2)

# Number of Unique Items
# Average Purchase Price
# Total Number of Purchases
# Total Revenue

pur_analysis = pd.DataFrame([{
    
    "Number of Unique Items": total_unique,
    'Average Purchase Price': avg_price,
    'Total Purchases': total_pur,
    'Total Revenue': total_rev
}])

pur_analysis

,Average Purchase Price,Number of Unique Items,Total Purchases,Total Revenue
0,2.93,183,780,2286.33


In [5]:
# Gender Demographics

# Percentage and Count of Male Players
# Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed

no_dup_players = pur_data.drop_duplicates(['SN'], keep ='last')
#player_count from above
gender_counts = no_dup_players['Gender'].value_counts().reset_index()
gender_counts['% of Players'] = gender_counts['Gender']/player_count * 100
gender_counts.rename(columns = {'index': 'Gender', 'Gender': '# of Players'}, inplace = True)
gender_counts.set_index('Gender')
#gender_counts['% of Players'].sum()
gender_counts.style.format({"% of Players": "{:.1f}%"})

,Gender,# of Players,% of Players
0,Male,465,81.2%
1,Female,100,17.5%
2,Other / Non-Disclosed,8,1.4%


In [6]:
# Purchasing Analysis (Gender)

# The below each broken by gender
# Purchase Count
# Average Purchase Price
# Total Purchase Value
# Normalized Totals

pur_count_by_gen = pd.DataFrame(pur_data.groupby('Gender')['Gender'].count())
total_pur_by_gen = pd.DataFrame(pur_data.groupby('Gender')['Price'].sum())
pur_analysis_gen = pd.merge(pur_count_by_gen, total_pur_by_gen, left_index = True, right_index = True)
pur_analysis_gen.rename(columns = {'Gender': '# of Purchases', 'Price':'Total Purchase Value'}, inplace=True)
pur_analysis_gen['Average Purchase Price'] = pur_analysis_gen['Total Purchase Value']/pur_analysis_gen['# of Purchases']
pur_analysis_gen = pur_analysis_gen.merge(gender_counts, left_index = True, right_on = 'Gender')
pur_analysis_gen['Normalized Totals'] = pur_analysis_gen['Total Purchase Value']/pur_analysis_gen['# of Players']
del pur_analysis_gen['% of Players']
del pur_analysis_gen['# of Players']
pur_analysis_gen.set_index('Gender', inplace=True)
pur_analysis_gen.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}', 'Normalized Totals': '${:.2f}'})

,# of Purchases,Total Purchase Value,Average Purchase Price,Normalized Totals
Gender,,,,
Female,136,$382.91,$2.82,$3.83
Male,633,$1867.68,$2.95,$4.02
Other / Non-Disclosed,11,$35.74,$3.25,$4.47


In [7]:
# The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
# Purchase Count
# Average Purchase Price
# Total Purchase Value
# Normalized Totals

pur_data.loc[(pur_data['Age'] < 10), 'age_bin'] = "< 10"
pur_data.loc[(pur_data['Age'] >= 10) & (pur_data['Age'] <= 14), 'age_bin'] = "10 - 14"
pur_data.loc[(pur_data['Age'] >= 15) & (pur_data['Age'] <= 19), 'age_bin'] = "15 - 19"
pur_data.loc[(pur_data['Age'] >= 20) & (pur_data['Age'] <= 24), 'age_bin'] = "20 - 24"
pur_data.loc[(pur_data['Age'] >= 25) & (pur_data['Age'] <= 29), 'age_bin'] = "25 - 29"
pur_data.loc[(pur_data['Age'] >= 30) & (pur_data['Age'] <= 34), 'age_bin'] = "30 - 34"
pur_data.loc[(pur_data['Age'] >= 35) & (pur_data['Age'] <= 39), 'age_bin'] = "35 - 39"
pur_data.loc[(pur_data['Age'] >= 40), 'age_bin'] = "> 40"
# pur_data[['age_bin', 'Age']].count()

pur_count_age = pd.DataFrame(pur_data.groupby('age_bin')['SN'].count())
avg_price_age = pd.DataFrame(pur_data.groupby('age_bin')['Price'].mean())
tot_pur_age = pd.DataFrame(pur_data.groupby('age_bin')['Price'].sum())
no_dup_age = pd.DataFrame(pur_data.drop_duplicates('SN', keep = 'last').groupby('age_bin')['SN'].count())
merge_age = pd.merge(pur_count_age, avg_price_age, left_index = True, right_index = True).merge(tot_pur_age, left_index = True, right_index = True).merge(no_dup_age, left_index = True, right_index = True)
merge_age.rename(columns = {"SN_x": "# of Purchases", "Price_x": "Average Purchase Price", "Price_y": "Total Purchase Value", "SN_y": "# of Purchasers"}, inplace = True)
merge_age['Normalized Totals'] = merge_age['Total Purchase Value']/merge_age['# of Purchasers']
merge_age.index.rename("Age", inplace = True)
merge_age

,# of Purchases,Average Purchase Price,Total Purchase Value,# of Purchasers,Normalized Totals
Age,,,,,
10 - 14,35,2.770000,96.95,23,4.215217
15 - 19,133,2.905414,386.42,100,3.864200
20 - 24,336,2.913006,978.77,259,3.779035
25 - 29,125,2.962640,370.33,87,4.256667
30 - 34,64,3.082031,197.25,47,4.196809
35 - 39,42,2.842857,119.40,27,4.422222
< 10,28,2.980714,83.46,19,4.392632
> 40,17,3.161765,53.75,11,4.886364


In [8]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
# SN
# Purchase Count
# Average Purchase Price
# Total Purchase Value

purchase_amt_by_SN = pd.DataFrame(pur_data.groupby('SN')['Price'].sum())
num_purchase_by_SN = pd.DataFrame(pur_data.groupby('SN')['Price'].count())
avg_purchase_by_SN = pd.DataFrame(pur_data.groupby('SN')['Price'].mean())
merged_top5 = pd.merge(purchase_amt_by_SN, num_purchase_by_SN, left_index = True, right_index = True).merge(avg_purchase_by_SN, left_index=True, right_index=True)
merged_top5.rename(columns = {'Price_x': 'Total Purchase Value', 'Price_y':'Purchase Count', 'Price':'Average Purchase Price'}, inplace = True)
merged_top5.sort_values('Total Purchase Value', ascending = False, inplace=True)
merged_top5 = merged_top5.head()
merged_top5.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}'})

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Undirrala66,$17.06,5,$3.41
Saedue76,$13.56,4,$3.39
Mindimnya67,$12.74,4,$3.18
Haellysu29,$12.73,3,$4.24
Eoda93,$11.58,3,$3.86


In [19]:
# Identify the 5 most popular items by purchase count, then list (in a table):
# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value

top5_items_ID = pd.DataFrame(pur_data.groupby('Item ID')['Item ID'].count())
top5_items_ID.sort_values('Item ID', ascending = False, inplace = True)
top5_items_ID = top5_items_ID.iloc[0:6][:]
top5_items_total = pd.DataFrame(pur_data.groupby('Item ID')['Price'].sum())
top5_items = pd.merge(top5_items_ID, top5_items_total, left_index = True, right_index = True)
no_dup_items = pur_data.drop_duplicates(['Item ID'], keep = 'last')
top5_merge_ID = pd.merge(top5_items, no_dup_items, left_index = True, right_on = 'Item ID')
top5_merge_ID = top5_merge_ID[['Item ID', 'Item Name', 'Item ID_x', 'Price_y', 'Price_x']]
top5_merge_ID.set_index(['Item ID'], inplace = True)
top5_merge_ID.rename(columns =  {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace=True)
top5_merge_ID.style.format({'Item Price': '${:.2f}', 'Total Purchase Value': '${:.2f}'})

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41
34,Retribution Axe,9,$4.14,$37.26


In [52]:
# Most Profitable Items

# Identify the 5 most profitable items by total purchase value, then list (in a table):
# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value

top5_profit = pd.DataFrame(pur_data.groupby('Item ID')['Price'].sum())
top5_profit.sort_values('Price', ascending = False, inplace = True)
top5_profit = top5_profit.iloc[0:5][:]
pur_count_profit = pd.DataFrame(pur_data.groupby('Item ID')['Item ID'].count())
top5_profit = pd.merge(top5_profit, pur_count_profit, left_index = True, right_index = True, how = 'left')
top5_merge_profit = pd.merge(top5_profit, no_dup_items, left_index = True, right_on = 'Item ID', how = 'left')
top5_merge_profit = top5_merge_profit[['Item ID', 'Item Name', 'Item ID_x', 'Price_y','Price_x']]
top5_merge_profit.set_index(['Item ID'], inplace=True)
top5_merge_profit.rename(columns = {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace = True)
top5_merge_profit.style.format({'Item Price': '${:.2f}', 'Total Purchase Value': '${:.2f}'})

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
